In [2]:
from protosc import filter_model
from protosc.simulation import create_simulation_data, create_correlated_data
from protosc.filter_model import train_xvalidate, select_features
from protosc.final_selection import final_selection
from protosc.wrapper import wrapper, calc_accuracy
from collections import defaultdict
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
# Create data: features (X) and categories (y)
np.random.seed(1928374)
X, y, ground_truth = create_correlated_data()
print(f'features: {X[:5]}')
print(f'categories: {y[:5]}')

features: [[ 1.35946304 -0.05495788 -1.25547915 ... -0.29250157  0.71268609
  -1.15811878]
 [ 1.16041638  0.11066386  0.96355358 ... -0.74329004  0.1515906
  -0.07321478]
 [ 0.30436521  0.61188844 -0.84293664 ... -0.04545197  1.07970547
  -1.35302247]
 [-0.44400432 -0.55848696  0.62296112 ...  0.09136625 -0.86405888
  -1.28561021]
 [-1.91110816 -2.27302481 -1.0157041  ... -0.01774011  0.13587279
  -0.88981648]]
categories: [0 0 0 1 1]


In [4]:
# Create clusters of correlating features, and select n features from these clusters for filter model
selected_clusters, clusters = select_features(X, y, chisq_threshold=0.25)
print(f'Clusters (n={len(clusters)}): {clusters[:10]}')
print(f'Selected features (n={len(selected_clusters)}): {selected_clusters}')

Clusters (n=200): [[485, 862, 788, 497, 473], [175, 81, 486, 344, 591], [635, 769, 682, 693, 863], [451, 251, 468, 176, 766], [597, 684, 12, 984, 950], [367, 983, 0, 215, 702], [191, 495, 60, 455, 261], [496, 955, 193, 903, 934], [479, 567, 107, 834, 511], [302, 124, 732, 899, 867]]
Selected features (n=20): [485, 862, 788, 497, 473, 175, 81, 486, 344, 591, 635, 769, 682, 693, 863, 451, 251, 468, 176, 766]


In [5]:
# Split data into training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Options for wrapper():
* X: np.array, features
* y: np.array, categories (1/0)
* clusters: np.array, clusters of correlating features
* decrease: boolean, if True clusters are ranked from high to low chi-square scores, if False from low to high
* add_im: boolean, if True clusters are immediately added to model if they increase the accuracy, if False it only adds the cluster with the highest accuracy increase
* search_space: float, percentage of clusters that will be used to select clusters from
* stop: int, max number of rounds where no clusters can be added, after which looping will stop
* n_fold: int, number of folds (used for calculating accuracy)

### 'Slow' method
Select clusters that increase accuracy, but per loop only add the cluster with *highest* impact.
* decrease: True
* add_im: False
* search_space: 0.15
* stop: 4
* n_fold: 3

In [6]:
model, selected, final_accuracy = wrapper(X, y, clusters, decrease=True, add_im=False, search_space=0.15, stop=4, n_fold=3)

IndexError: invalid index to scalar variable.

In [ ]:
print(f'Selected clusters = {selected}')
print(f'Selected features = {model}')
print(f'Final accuracy = {final_accuracy}')

### 'Fast' method
Select clusters that increase accuracy; when one cluster increases accuracy, immediately add it to model and continue with this new model
* decrease: True
* add_im: True
* search_space: 0.15
* stop: 4
* n_fold: 3

In [ ]:
model_f, selected_f, final_accuracy_f = wrapper(X, y, clusters, decrease=True, add_im=True, search_space=0.15, stop=4, n_fold=3)

In [ ]:
print(f'Selected clusters = {selected_f}')
print(f'Selected features = {model_f}')
print(f'Final accuracy = {final_accuracy_f}')